In [ ]:
# --- Import di base ---
import json
import os
import pandas as pd
import numpy as np
from collections import Counter
import math
from tqdm.notebook import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# --- Import Modelli ---
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier # Aggiunto kNN
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.feature_builder_Model1 import *


from src.config_Model1 import *

try:
    from src.data_processing import load_jsonl_data, clean_raw_data
except ImportError:
    print("ATTENZIONE: 'load_jsonl_data' non trovato in src.data_processing.py. Verrà usata la logica locale.")
    
    # Definizione locale
    def load_jsonl_data(file_path: str) -> pd.DataFrame:
        data = []
        with open(file_path, 'r') as f:
            for line in f:
                data.append(json.loads(line))
        return pd.DataFrame(data)

    def clean_raw_data(df: pd.DataFrame) -> pd.DataFrame:
        df_cleaned = df.copy()
        ROW_TO_DROP = 4877
        if ROW_TO_DROP in df_cleaned.index:
            df_cleaned = df_cleaned.drop(index=ROW_TO_DROP)
        
        print("Eseguita pulizia dati locale.")
        return df_cleaned.reset_index(drop=True)


# Impostazioni
warnings.filterwarnings('ignore')
SEED = 123
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'

print("Librerie e moduli per Model1 importati.")

In [ ]:
print("Caricamento e pulizia dati di training...")

# Path
BASE_DATA_PATH = os.path.join('..', 'data', 'raw')
TRAIN_FILE_PATH = os.path.join(BASE_DATA_PATH, 'train.jsonl')

# Carica
df_raw_train = load_jsonl_data(TRAIN_FILE_PATH)

# Pulisci
df_train_cleaned = clean_raw_data(df_raw_train)

# Mescola
df_train_shuffled = df_train_cleaned.sample(frac=1, random_state=SEED).reset_index(drop=True)

print(f"Dati di training pronti. Shape: {df_train_shuffled.shape}")

In [ ]:
print("--- Generazione Feature Set v8, v19, v20 ---")

# Estrae i set di feature usando le funzioni importate
X_train_v8, y_train = build_feature_dataframe(df_train_shuffled, extract_features_v8, is_test_set=False)
X_train_v20, _ = build_feature_dataframe(df_train_shuffled, extract_features_v20, is_test_set=False)
X_train_v19, _ = build_feature_dataframe(df_train_shuffled, extract_features_v19, is_test_set=False)

print("\n--- Shape dei Feature Set Creati ---")
print(f"X_train_v8 (per LR-v8):        {X_train_v8.shape}")
print(f"X_train_v20 (per XGB):         {X_train_v20.shape}")
print(f"X_train_v19 (per RF/CAT/kNN): {X_train_v19.shape}")
print(f"y_train (Target):              {y_train.shape}")

In [ ]:
print("\n--- Salvataggio Feature Set in data/processed/ ---")

PROCESSED_PATH = os.path.join('..', 'data', 'processed')
os.makedirs(PROCESSED_PATH, exist_ok=True)

# Salva V8
X_train_v8.to_csv(os.path.join(PROCESSED_PATH, 'v8_train_features.csv'), index=False)
print("Salvataggio v8_train_features.csv completato.")

# Salva V20
X_train_v20.to_csv(os.path.join(PROCESSED_PATH, 'v20_train_features.csv'), index=False)
print("Salvataggio v20_train_features.csv completato.")

# Salva V19
X_train_v19.to_csv(os.path.join(PROCESSED_PATH, 'v19_train_features.csv'), index=False)
print("Salvataggio v19_train_features.csv completato.")

# Salva il Target
y_train_df = y_train.to_frame(name='player_won')
y_train_df.to_csv(os.path.join(PROCESSED_PATH, 'train_target.csv'), index=False)
print("Salvataggio train_target.csv completato.")

print("\nDone! I file sono pronti per '02_All_Base_Models_Training.ipynb'.")

In [ ]:
# Contenitore per i nostri modelli base
base_models = {}

# === 1. Modello LR (v8) ===
# Da: prova-xg-vs-logistic-v2.ipynb
model_lr_v8 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        C=10.0, 
        penalty='l2', 
        solver='saga', 
        max_iter=5000, 
        random_state=SEED
    ))
])
base_models['lr_v8'] = (model_lr_v8, X_train_v8)

# === 2. Modello XGB (v20) ===
# Da: xg-vs-logit-con-switch-strategy.ipynb
model_xgb_v20 = XGBClassifier(
    colsample_bytree=0.7,
    learning_rate=0.05,
    max_depth=3,
    n_estimators=200,
    reg_lambda=5,
    subsample=0.7,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=SEED
)
base_models['xgb_v20'] = (model_xgb_v20, X_train_v20)

# === 3. Modello RF (v19) ===
# Da: random-forrest.ipynb
model_rf_v19 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(
        n_estimators=400,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features=0.5,
        max_depth=10,
        random_state=SEED,
        n_jobs=-1
    ))
])
base_models['rf_v19'] = (model_rf_v19, X_train_v19)

# === 4. Modello CAT (v19) ===
# Da: knn-vs-catboost.ipynb
model_cat_v19 = CatBoostClassifier(
    learning_rate=0.03,
    l2_leaf_reg=7,
    iterations=300,
    depth=8,
    random_state=SEED,
    verbose=0,
    eval_metric='Accuracy'
)
base_models['cat_v19'] = (model_cat_v19, X_train_v19)

# === 5. Modello kNN (v19) ===
# Da: knn-vs-catboost.ipynb
model_knn_v19 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', KNeighborsClassifier(
        metric='manhattan',
        n_neighbors=45,
        weights='uniform',
        n_jobs=-1
    ))
])
base_models['knn_v19'] = (model_knn_v19, X_train_v19)


print(f"Definiti {len(base_models)} modelli base pronti per lo stacking.")
print(f"Modelli nello stack: {list(base_models.keys())}")

In [ ]:
N_SPLITS = 5
kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Creiamo il DataFrame X_meta per le OOF predictions
oof_preds = np.zeros((len(y_train), len(base_models)))
X_meta_df = pd.DataFrame(oof_preds, columns=base_models.keys())

# Modelli che serviranno per la submission (addestrati su tutto)
final_base_models = {}

print(f"Avvio Stacking (OOF) con {N_SPLITS} folds...")

# Usiamo tqdm per una barra di progresso
for fold, (train_idx, val_idx) in enumerate(tqdm(kfold.split(y_train, y_train), total=N_SPLITS, desc="Folds")):
    
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    for name, (model, X_data) in base_models.items():
        # Prendi i dati di training/validazione per *questo* set di feature
        X_train_fold = X_data.iloc[train_idx]
        X_val_fold = X_data.iloc[val_idx]
        
        # Addestra il modello
        model.fit(X_train_fold, y_train_fold)
        
        # Salva le previsioni OOF (probabilità)
        X_meta_df.loc[val_idx, name] = model.predict_proba(X_val_fold)[:, 1]

print("\nCreazione Meta-Features (X_meta_df) completata.")

# Ora, addestra i modelli base sull'INTERO dataset di training
# Ci serviranno per predire sul test set
print("Addestramento modelli base finali su tutti i dati di training...")
for name, (model, X_data) in tqdm(base_models.items(), desc="Modelli Finali"):
    final_base_models[name] = model.fit(X_data, y_train)

print("Modelli base finali addestrati.")
display(X_meta_df.head())

In [ ]:
print("--- Correlazione delle Previsioni OOF (Meta-Features) ---")

plt.figure(figsize=(10, 7))
sns.heatmap(
    X_meta_df.corr(), 
    annot=True, 
    cmap='coolwarm', 
    fmt=".3f"
)
plt.title(f"Correlazione Modelli Base (Stack {list(base_models.keys())})")
plt.show()

print("Commento: Cerchiamo valori più bassi possibile.")
print("Il kNN dovrebbe avere una correlazione più bassa con gli altri, il che è ottimo per l'ensemble.")

In [ ]:
"""
ANALISI ENSEMBLE - SELEZIONE MODELLI OTTIMALE
==============================================

Questo codice va inserito DOPO la cella 8 (dopo aver creato X_meta_df)
e PRIMA della cella 9 (addestramento meta-modello finale)
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

print("="*80)
print("ANALISI ENSEMBLE - SELEZIONE MODELLI")
print("="*80)

# ============================================================================
# METODO 1: PERFORMANCE INDIVIDUALI (OOF)
# ============================================================================
print("\n" + "="*80)
print("METODO 1: PERFORMANCE INDIVIDUALI DEI MODELLI BASE")
print("="*80)

# Calcola accuracy OOF per ogni modello singolarmente
individual_scores = {}
for name in X_meta_df.columns:
    # Predici con threshold 0.5
    predictions = (X_meta_df[name] > 0.5).astype(int)
    accuracy = (predictions == y_train).mean()
    individual_scores[name] = accuracy
    
# Ordina per performance
sorted_scores = dict(sorted(individual_scores.items(), key=lambda x: x[1], reverse=True))

print("\nACCURACY OOF INDIVIDUALE:")
print("-" * 50)
for name, score in sorted_scores.items():
    print(f"{name:10s}: {score:.4f}")

# Visualizza
plt.figure(figsize=(10, 5))
plt.barh(list(sorted_scores.keys()), list(sorted_scores.values()), color='steelblue')
plt.xlabel('OOF Accuracy')
plt.title('Performance Individuale Modelli Base')
plt.xlim([min(sorted_scores.values())-0.01, max(sorted_scores.values())+0.01])
for i, (name, score) in enumerate(sorted_scores.items()):
    plt.text(score, i, f' {score:.4f}', va='center')
plt.tight_layout()
plt.show()

# ============================================================================
# METODO 2: CORRELAZIONE PREDIZIONI (GIÀ HAI LA HEATMAP)
# ============================================================================
print("\n" + "="*80)
print("METODO 2: ANALISI CORRELAZIONE PREDIZIONI")
print("="*80)

correlation_matrix = X_meta_df.corr()
print("\nMATRICE DI CORRELAZIONE:")
print(correlation_matrix.round(3))

# Trova coppie ad alta correlazione (>0.9 = ridondanti)
print("\n⚠️  COPPIE AD ALTA CORRELAZIONE (>0.90 - Potenziale Ridondanza):")
print("-" * 50)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if corr_value > 0.90:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j], corr_value)
            high_corr_pairs.append(pair)
            print(f"{pair[0]:10s} <-> {pair[1]:10s}: {pair[2]:.4f}")

if not high_corr_pairs:
    print("✓ Nessuna coppia con correlazione >0.90 (Buona diversità!)")

# Media correlazione per modello (quanto è simile agli altri)
avg_corr = {}
for col in correlation_matrix.columns:
    # Media correlazioni con altri modelli (escluso se stesso)
    other_corrs = correlation_matrix[col].drop(col)
    avg_corr[col] = other_corrs.mean()

print("\n📊 MEDIA CORRELAZIONE CON ALTRI MODELLI:")
print("-" * 50)
for name, avg in sorted(avg_corr.items(), key=lambda x: x[1]):
    print(f"{name:10s}: {avg:.4f} {'⭐ (Più diverso)' if avg == min(avg_corr.values()) else ''}")

# ============================================================================
# METODO 3: BACKWARD ELIMINATION (Rimuovi uno alla volta)
# ============================================================================
print("\n" + "="*80)
print("METODO 3: BACKWARD ELIMINATION")
print("="*80)
print("(Rimuove modelli uno alla volta, verifica impatto su CV score)\n")

meta_model = LogisticRegression(random_state=SEED, max_iter=1000)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Score baseline (tutti i modelli)
baseline_score = cross_val_score(
    meta_model, X_meta_df, y_train, cv=kfold, scoring='accuracy', n_jobs=-1
).mean()

print(f"📌 BASELINE (Tutti i {len(X_meta_df.columns)} modelli): {baseline_score:.4f}")
print("-" * 50)

# Prova a rimuovere ogni modello singolarmente
removal_impact = {}
for col_to_remove in X_meta_df.columns:
    X_reduced = X_meta_df.drop(columns=[col_to_remove])
    score = cross_val_score(
        meta_model, X_reduced, y_train, cv=kfold, scoring='accuracy', n_jobs=-1
    ).mean()
    impact = score - baseline_score
    removal_impact[col_to_remove] = {'score': score, 'impact': impact}
    
    emoji = "📉" if impact < 0 else "📈" if impact > 0 else "➡️"
    print(f"{emoji} Senza {col_to_remove:10s}: {score:.4f} (Δ = {impact:+.4f})")

# Trova il modello la cui rimozione danneggia meno (o migliora)
least_damaging = max(removal_impact.items(), key=lambda x: x[1]['impact'])
print(f"\n💡 CANDIDATO ALLA RIMOZIONE: {least_damaging[0]}")
print(f"   Score senza: {least_damaging[1]['score']:.4f}")
print(f"   Impatto: {least_damaging[1]['impact']:+.4f}")

if least_damaging[1]['impact'] >= 0:
    print(f"   ✓ Rimuoverlo MIGLIORA o non peggiora il modello!")
else:
    print(f"   ⚠️  Rimuoverlo peggiora il modello di {abs(least_damaging[1]['impact']):.4f}")

# ============================================================================
# METODO 4: FORWARD SELECTION (Costruisci dal migliore)
# ============================================================================
print("\n" + "="*80)
print("METODO 4: FORWARD SELECTION")
print("="*80)
print("(Parte dal modello migliore, aggiunge uno alla volta quello che migliora di più)\n")

# Ordina modelli per performance individuale
sorted_models = sorted(individual_scores.items(), key=lambda x: x[1], reverse=True)

# Inizia col migliore
selected = [sorted_models[0][0]]
remaining = [m[0] for m in sorted_models[1:]]

print(f"📌 INIZIO CON: {selected[0]} (Accuracy: {sorted_models[0][1]:.4f})")
print("-" * 50)

forward_history = []
current_score = cross_val_score(
    meta_model, X_meta_df[selected], y_train, cv=kfold, scoring='accuracy', n_jobs=-1
).mean()
forward_history.append({'models': selected.copy(), 'score': current_score})

print(f"CV Score con [{', '.join(selected)}]: {current_score:.4f}\n")

# Aggiungi modelli uno alla volta
while remaining:
    best_addition = None
    best_score = current_score
    
    for candidate in remaining:
        test_set = selected + [candidate]
        score = cross_val_score(
            meta_model, X_meta_df[test_set], y_train, cv=kfold, scoring='accuracy', n_jobs=-1
        ).mean()
        
        if score > best_score:
            best_score = score
            best_addition = candidate
    
    if best_addition is not None:
        selected.append(best_addition)
        remaining.remove(best_addition)
        current_score = best_score
        forward_history.append({'models': selected.copy(), 'score': current_score})
        
        improvement = current_score - forward_history[-2]['score']
        print(f"➕ Aggiunto {best_addition:10s}: {current_score:.4f} (Δ = +{improvement:.4f})")
    else:
        print(f"\n⛔ STOP: Nessun modello migliora ulteriormente il CV score")
        break

print(f"\n🏆 MIGLIOR COMBINAZIONE (Forward): {selected}")
print(f"   CV Score: {current_score:.4f}")

# Visualizza storia Forward Selection
fig, ax = plt.subplots(figsize=(10, 5))
scores = [h['score'] for h in forward_history]
labels = [f"{i+1}: {', '.join(h['models'][:2])}..." if len(h['models']) > 2 
          else f"{i+1}: {', '.join(h['models'])}" 
          for i, h in enumerate(forward_history)]
ax.plot(range(1, len(scores)+1), scores, marker='o', linewidth=2, markersize=8)
ax.set_xlabel('Numero di Modelli nell\'Ensemble')
ax.set_ylabel('CV Accuracy')
ax.set_title('Forward Selection: Andamento CV Score')
ax.set_xticks(range(1, len(scores)+1))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.grid(alpha=0.3)
ax.axhline(baseline_score, color='red', linestyle='--', label=f'Baseline (tutti): {baseline_score:.4f}')
ax.legend()
plt.tight_layout()
plt.show()

# ============================================================================
# METODO 5: PROVA TUTTE LE COMBINAZIONI (Brute Force - Per 5 modelli è fattibile)
# ============================================================================
print("\n" + "="*80)
print("METODO 5: VALUTAZIONE TUTTE LE COMBINAZIONI")
print("="*80)
print(f"(Con {len(X_meta_df.columns)} modelli ci sono {2**len(X_meta_df.columns)-1} combinazioni possibili)\n")

all_models = list(X_meta_df.columns)
all_combinations = []

# Prova tutte le combinazioni da 1 a N modelli
for size in range(1, len(all_models) + 1):
    for combo in combinations(all_models, size):
        score = cross_val_score(
            meta_model, X_meta_df[list(combo)], y_train, cv=kfold, scoring='accuracy', n_jobs=-1
        ).mean()
        all_combinations.append({
            'models': list(combo),
            'n_models': len(combo),
            'score': score
        })

# Ordina per score
all_combinations_sorted = sorted(all_combinations, key=lambda x: x['score'], reverse=True)

print("🏆 TOP 10 COMBINAZIONI:")
print("-" * 80)
for i, combo in enumerate(all_combinations_sorted[:10], 1):
    models_str = ', '.join(combo['models'])
    print(f"{i:2d}. [{combo['n_models']} modelli] {combo['score']:.4f} - {models_str}")

# Migliore per ogni numero di modelli
print("\n📊 MIGLIOR COMBINAZIONE PER NUMERO DI MODELLI:")
print("-" * 80)
best_by_size = {}
for combo in all_combinations:
    size = combo['n_models']
    if size not in best_by_size or combo['score'] > best_by_size[size]['score']:
        best_by_size[size] = combo

for size in sorted(best_by_size.keys()):
    combo = best_by_size[size]
    models_str = ', '.join(combo['models'])
    print(f"{size} modelli: {combo['score']:.4f} - [{models_str}]")

# Visualizza
fig, ax = plt.subplots(figsize=(12, 6))
for size in sorted(best_by_size.keys()):
    scores_for_size = [c['score'] for c in all_combinations if c['n_models'] == size]
    ax.scatter([size] * len(scores_for_size), scores_for_size, alpha=0.3, s=50)
    
# Linea dei migliori
best_scores = [best_by_size[s]['score'] for s in sorted(best_by_size.keys())]
ax.plot(sorted(best_by_size.keys()), best_scores, 'ro-', linewidth=2, markersize=10, label='Best per size')
ax.axhline(baseline_score, color='green', linestyle='--', linewidth=2, label=f'Baseline (tutti): {baseline_score:.4f}')
ax.set_xlabel('Numero di Modelli nell\'Ensemble')
ax.set_ylabel('CV Accuracy')
ax.set_title('Tutte le Combinazioni: CV Score vs Numero di Modelli')
ax.set_xticks(sorted(best_by_size.keys()))
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# RIEPILOGO FINALE
# ============================================================================
print("\n" + "="*80)
print("📋 RIEPILOGO E RACCOMANDAZIONI")
print("="*80)

print(f"\n1️⃣  BASELINE (Tutti i {len(X_meta_df.columns)} modelli):")
print(f"   CV Score: {baseline_score:.4f}")

print(f"\n2️⃣  BACKWARD ELIMINATION suggerisce:")
print(f"   Rimuovere: {least_damaging[0]}")
print(f"   Score risultante: {least_damaging[1]['score']:.4f}")

print(f"\n3️⃣  FORWARD SELECTION suggerisce:")
print(f"   Modelli: {selected}")
print(f"   Score: {current_score:.4f}")

print(f"\n4️⃣  MIGLIOR COMBINAZIONE ASSOLUTA (Brute Force):")
best_overall = all_combinations_sorted[0]
print(f"   Modelli: {best_overall['models']}")
print(f"   Score: {best_overall['score']:.4f}")

# Confronto con baseline
improvement = best_overall['score'] - baseline_score
if improvement > 0.0005:  # Miglioramento significativo
    print(f"\n✅ RACCOMANDAZIONE: Usa la combinazione ottimale trovata")
    print(f"   Miglioramento: +{improvement:.4f}")
    print(f"   Modelli da usare: {best_overall['models']}")
elif improvement < -0.0005:  # Peggioramento
    print(f"\n⚠️  RACCOMANDAZIONE: Mantieni tutti i modelli (baseline)")
    print(f"   La combinazione ottimale è peggiore: {improvement:.4f}")
else:  # Differenza trascurabile
    print(f"\n➡️  RACCOMANDAZIONE: Baseline vs Ottimale sono equivalenti")
    print(f"   Differenza trascurabile: {improvement:.4f}")
    if len(best_overall['models']) < len(X_meta_df.columns):
        print(f"   Suggerisco: Usa {best_overall['models']} (più semplice)")
    else:
        print(f"   Suggerisco: Mantieni tutti (più robusto)")

print("\n" + "="*80)
print("💡 SUGGERIMENTI AGGIUNTIVI:")
print("="*80)
print("- Se hai modelli con correlazione >0.95, considera di rimuovere il più debole")
print("- kNN spesso ha bassa correlazione ma può essere debole individualmente")
print("- XGBoost e CatBoost tendono ad essere correlati (entrambi gradient boosting)")
print("- Il meta-modello (LogReg) può dare pesi diversi ai modelli automaticamente")
print("- Se il dataset è piccolo, meno modelli = meno overfitting del meta-modello")
print("="*80)

# ============================================================================
# CREA SUBSET OTTIMALE PER PROSSIMI STEP
# ============================================================================
print(f"\n🔧 Creazione X_meta_df ottimizzato...")
optimal_models = best_overall['models']
X_meta_df_optimal = X_meta_df[optimal_models].copy()
print(f"   X_meta_df_optimal creato con modelli: {optimal_models}")
print(f"   Usa 'X_meta_df_optimal' nella cella successiva per il meta-modello finale")
print("="*80)

In [ ]:
print("--- Addestramento e Valutazione Meta-Modello (Livello 1) ---")

# Un modello semplice è la scelta migliore per il meta-modello.
meta_model = LogisticRegression(random_state=SEED)

# Valutiamo il nostro ensemble finale usando la CV
# Questo ci dà lo score più onesto
cv_scores = cross_val_score(
    meta_model, 
    X_meta_df_optimal,  # Le nostre feature di Livello 1 (le previsioni OOF)
    y_train,    # I target reali
    cv=kfold, 
    scoring='accuracy',
    n_jobs=-1
)

print(f"\nScore CV dell'Ensemble Finale (Stima Onesta):")
print(f"Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
print(f"Scores dei Fold: {[round(s, 4) for s in cv_scores]}")


# Addestra il meta-modello finale sul SET OTTIMALE di meta-feature OOF
print(f"Addestramento del Meta-Modello finale su X_meta_df_optimal ({X_meta_df_optimal.shape})...")
final_ensemble_model = meta_model.fit(X_meta_df_optimal, y_train)
print("Meta-Modello pronto per la submission!")

In [ ]:
print("--- Preparazione Dati di Test per la Submission ---")

test_file_path = os.path.join(DATA_PATH, 'test.jsonl')
test_data_raw = []

print(f"Caricamento dati di TEST da '{test_file_path}'...")
with open(test_file_path, 'r') as f:
    for line in f:
        test_data_raw.append(json.loads(line))
df_test_raw = pd.DataFrame(test_data_raw) 

# --- Applica TUTTE le Feature Engineering ---
print("Generazione di tutti i Feature Set di Test...")

X_test_v8  = build_feature_dataframe(df_test_raw, extract_features_v8, is_test_set=True)
X_test_v20 = build_feature_dataframe(df_test_raw, extract_features_v20, is_test_set=True)
X_test_v19 = build_feature_dataframe(df_test_raw, extract_features_v19, is_test_set=True)

print("\n--- Shape dei Feature Set di Test ---")
print(f"X_test_v8:  {X_test_v8.shape}")
print(f"X_test_v20: {X_test_v20.shape}")
print(f"X_test_v19: {X_test_v19.shape}")

In [ ]:
print("--- Creazione Meta-Features di Test ---")

# Dizionario per i dati di test
test_data_map = {
    'lr_v8': X_test_v8,
    'xgb_v20': X_test_v20,
    'rf_v19': X_test_v19,
    'cat_v19': X_test_v19,
    'knn_v19': X_test_v19  # kNN usa le feature v19
}

# DataFrame vuoto per le previsioni di test
X_meta_test_df = pd.DataFrame(columns=base_models.keys())

for name, model in tqdm(final_base_models.items(), desc="Predizioni Test L0"):
    # Prendi il set di feature di test corrispondente
    X_test_data = test_data_map[name]
    
    # Predici le probabilità
    X_meta_test_df[name] = model.predict_proba(X_test_data)[:, 1]

print("Meta-Features di Test create.")
display(X_meta_test_df.head())

In [ ]:
print("--- Generazione Submission Finale ---")
# Filtra le meta-feature di test per usare solo i modelli ottimali
# (La variabile 'optimal_models' è stata definita nella Cella 9)
print(f"Filtraggio delle meta-feature di test sui modelli ottimali: {optimal_models}")
X_meta_test_optimal_SUB = X_meta_test_df[optimal_models]
# Usa il meta-modello addestrato (Cella 8) per predire sulle meta-feature di test
final_predictions = final_ensemble_model.predict(X_meta_test_optimal_SUB)
print("Previsioni finali generate.")

# --- Creazione File Submission ---
submission_df = pd.DataFrame({
    'battle_id': df_test_raw['battle_id'],
    'player_won': final_predictions.astype(int)
})

submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"File '{submission_filename}' creato con successo!")
print("In bocca al lupo per la competizione!")
display(submission_df.head())